In [35]:
import deepspeech
from datasets import load_dataset
import numpy as np
import pandas as pd
import soundfile as sf
import time


In [36]:
PATH_TRANSCRIPTIONS = "./../eval/transcriptions.csv"
pre_train_file = 'deepspeech-0.9.3-models.pbmm'
beam_width = 300
model = deepspeech.Model(model_file_path)

In [38]:
# define function to read in sound file
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch
    
# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

Reusing dataset librispeech_asr (/home/lucasagrizzi/.cache/huggingface/datasets/patrickvonplaten___librispeech_asr/clean/2.1.0/f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc)
Loading cached processed dataset at /home/lucasagrizzi/.cache/huggingface/datasets/patrickvonplaten___librispeech_asr/clean/2.1.0/f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc/cache-361ab9f8e626e5d4.arrow


In [64]:

# initialize constants
transcriptions = []
error = []
inference_time = []

len_ds = len(ds["text"])
BATCH_SIZE = 1


for i in range(len_ds//BATCH_SIZE):

    # import audio
    audio = ds["speech"][i]
    audio = np.array(audio)
    a = audio + np.abs(audio.min())
    a = (a * 1024).astype(np.int16)
    # count time of inference
    start = time.time()
    
    transcription = model.stt(a)

    # store time of inference
    inference_time.append(time.time() - start)

    # store transcription
    transcriptions.append(transcription)

all_transcriptions = list(np.array(transcriptions).reshape(-1))

In [65]:
df_transcriptions = pd.read_csv(PATH_TRANSCRIPTIONS, index_col=0)

df_transcriptions[pre_train_file] = all_transcriptions
df_transcriptions[pre_train_file] = df_transcriptions[pre_train_file].apply(lambda x: x.upper())

df_transcriptions[pre_train_file + "_inf_time"] = inference_time
df_transcriptions

,ground_truth,facebook/wav2vec2-large-960h-lv60-self,facebook/wav2vec2-large-960h-lv60-self_inf_time,facebook/s2t-large-librispeech-asr,facebook/s2t-large-librispeech-asr_inf_time,speechbrain/asr-crdnn-rnnlm-librispeech,speechbrain/asr-crdnn-rnnlm-librispeech_inf_time,deepspeech-0.9.3-models.pbmm,deepspeech-0.9.3-models.pbmm_inf_time
0,A MAN SAID TO THE UNIVERSE SIR I EXIST,A MAN SAID TO THE UNIVERSE SIR I EXIST,5.948975,A MAN SAID TO THE UNIVERSE SIR I EXIST,28.164044,A MAN SAID TO THE UNIVERSE SIR I EXIST,7.624899,BECAUSE YOU WRE ASLEEPING IN SOME OF CONQUERIN...,9.392330
1,SWEAT COVERED BRION'S BODY TRICKLING INTO THE ...,SWEAT COVERED BRION'S BODY TRICKLING INTO THE ...,7.317595,SWEAT COVERED BRION'S BODY TRICKLING INTO THE ...,21.338112,SWEAT COVERED BRIAN'S BODY TRICKLING INTO THE ...,22.286610,HE HAS GONE AND GONE FOR GOOD ANSWERED PALACRO...,9.637827
2,THE CUT ON HIS CHEST STILL DRIPPING BLOOD THE ...,THE CUT ON HIS CHEST STILL DRIPPING BLOOD THE ...,14.338364,THE CUT ON HIS CHEST STILL DRIPPING BLOOD THE ...,32.019719,THE CUT ON HIS CHEST STILL DRIPPING BLOOD THE ...,29.881027,I HAVE REMAINED A PRESONER ONLY BECAUSE I WISH...,10.651164
3,HIS INSTANT OF PANIC WAS FOLLOWED BY A SMALL S...,HIS INSTANT PANIC WAS FOLLOWED BY A SMALL SHAR...,6.099911,HIS INSTANT PANIC WAS FOLLOWED BY A SMALL SHAR...,14.342224,HIS INSTANT PANIC WAS FOLLOWED BY A SMALL SHAR...,10.652006,THE LITTLE GIRL HAD BEEN ASLEEP BUT SHE HEARD ...,4.103545
4,ONE MINUTE A VOICE SAID AND THE TIME BUZZER SO...,ONE MINUTE A VOICE SAID AND THE TIME BUZZER SO...,5.535601,ONE MINUTE A VOICE SAID AND A TIME BUZZER SOUNDED,12.898235,ONE MINUTE A VOICE SAID AND THE TIME BUZZER SO...,8.057678,THE KING IS WORNE DISGRACE IN YOUR FRIENDS OR ...,3.449225
...,...,...,...,...,...,...,...,...,...
68,I DON'T BELIEVE ANN KNEW ANY MAGIC OR SHE'D HA...,I DON'T BELIEVE ANNE KNEW ANY MAGIC OR SHE'D H...,12.336592,I DON'T BELIEVE ANNE KNEW ANY MAGIC OR SHE'D H...,18.082567,I DON'T BELIEVE ANNE KNEW ANY MAGIC OR SEED OF...,10.486318,A ROE LOOKED THE MAZED AT THE SUDDEN FURY OF T...,4.648898
69,I DO NOT KNOW CONFESSED SHAGGY,I DO NOT KNOW CONFESSED SHAGGY,19.080031,I DO NOT KNOW CONFESSED SHAGGY,22.220248,I DO NOT KNOW CONFESSED SHAGGY,6.238551,HE ANTHER WAS THE LAST BURST OF VENERGY HE KNE...,5.569380
70,TRUE AGREED KALIKO,TRUE A GREEN CALICO,15.415310,TRUE AGREED KALIKO,30.322897,TRUE AGREED CALICO,4.310126,BRE UNSA SOMETHING CLOSED PANIC ON HIS UPON AN...,5.793082
71,KALIKO WENT TO THE BIG GONG AND POUNDED ON IT ...,CALICO WENT TO THE BIG GONG AND POUNDED ON IT ...,18.959109,KALIKO WENT TO THE BIG GONG AND POUNDED ON IT ...,2244.000314,CALICO WENT TO THE BIG GONG AND POUNDED ON IT ...,17.696929,AWAVE OF THISPAR ROLED OUTE FROM IROLG BREAN S...,5.546355


In [ ]:
df_transcriptions.to_csv(PATH_TRANSCRIPTIONS)

In [19]:
# import numpy as np
# import wave
# filename = 'example.wav'
# w = wave.open(filename, 'r')
# rate = w.getframerate()
# frames = w.getnframes()
# buffer = w.readframes(frames)
# print(rate)
# print(model.sampleRate())
# type(buffer)
# data16 = np.frombuffer(buffer, dtype=np.int16)
# type(data16)
# print(data16.shape)
# text = model.stt(data16)
# print(text)

(78480,)
